### Objective 1
We want something which will, given the md trajectory, calculate the dihedral angles (from this we can get their distributions)
given the relevant SMILES string.

### Objective 2
We also want something to plot the csv file data, so that we can do a sanity check to ensure the macroscopic properties (temp, pressure)
are reasonable and are not discontinuous.

In [15]:
import os
from typing import List

from openmm import *
from openmm.app import *

import mdtraj as md
import matplotlib.pyplot as plt
import numpy as np
import openmoltools
import tempfile
import cctk

from rdkit import Chem
from rdkit.Chem import AllChem

In [23]:
smiles_dict = {
    'Lys-Tyr': 'CNCc1c(O)ccc(c1)C',
    'Lys-Arg': 'N1(C)CN=C(NC1)NC',
    'Sulfur-Mediated-Amide': 'SC[C@@H](NC(=O)C)C(=O)C',
    'Carboxyl-Carboxyl': 'C(=O)(C)NCc1ccc(cc1)CNC(=O)C',
    'Disulfide': 'N[C@H](C(=O)O)CSSC[C@@H](N)C(=O)O',
    'Cys-Arg': 'CSCC(=O)NCCCC',
    'Cys-Carboxyl': 'CSCC(=O)C',
}

def generate_initial_pdb(
    smiles: str,
    min_side_length: int = 25, # Å
    solvent_smiles = "O",
) -> str:
    """ Creates a PDB file for a solvated molecule, starting from two SMILES strings. """

    # do some math to figure how big the box needs to be
    solute = cctk.Molecule.new_from_smiles(smiles)
    solute_volume = solute.volume(qhull=True)
    solvent = cctk.Molecule.new_from_smiles(solvent_smiles)
    solvent_volume = solvent.volume(qhull=False)

    total_volume = 50 * solute_volume # seems safe?
    min_allowed_volume = min_side_length ** 3
    total_volume = max(min_allowed_volume, total_volume)

    total_solvent_volume = total_volume - solute_volume
    n_solvent = int(total_solvent_volume // solvent_volume)
    box_size = total_volume ** (1/3)

    # build pdb
    tempdir = tempfile.mkdtemp()
    system_fname = os.path.join(tempdir, "system.pdb")
    
    solute_fname = os.path.join(tempdir, "solute.pdb")
    solvent_fname = os.path.join(tempdir, "solvent.pdb")

    smiles_to_pdb(smiles, solute_fname)
    smiles_to_pdb(solvent_smiles, solvent_fname)
    traj_packmol = openmoltools.packmol.pack_box(
      [solute_fname, solvent_fname],
      [1, n_solvent],
      box_size=box_size
     )
    traj_packmol.save_pdb(system_fname)

    return system_fname

def smiles_to_pdb(smiles: str, filename: str) -> None:
    """ Turns a SMILES string into a PDB file (written to current working directory). """
    m = Chem.MolFromSmiles(smiles)
    mh = Chem.AddHs(m)
    AllChem.EmbedMolecule(mh)
    Chem.MolToPDBFile(mh, filename)

def load_trajectory(traj_path: str, smiles: str, fraction: float=0.2) -> md.Trajectory:
    """Load an MD trajectory from a .dcd file using a generated .pdb file for topology.
    Only keeps the last `fraction` of the frames."""
    
    # Generate the correct PDB file from SMILES
    pdb_path = generate_initial_pdb(smiles)
    
    # Load the topology from the generated PDB file
    pdb_topology = md.load(pdb_path).topology
    
    # Print the expected number of atoms from the PDB
    print(f"Expected number of atoms from PDB: {pdb_topology.n_atoms}")

    # Load the DCD trajectory
    traj = md.load(traj_path)
    
    # Print the actual number of atoms in the DCD file
    print(f"Number of atoms in DCD file: {traj.topology.n_atoms}")
    
    # Load the DCD trajectory
    traj = md.load(traj_path, top=pdb_path)
    
    # Print the actual number of atoms from the DCD file
    print(f"Number of atoms in DCD file: {traj.topology.n_atoms}")
    
    # Debugging: If mismatch, print first few atom names
    if traj.topology.n_atoms != pdb_topology.n_atoms:
        print("Mismatch detected! First few atoms in PDB:")
        for atom in pdb_topology.atoms[:10]:
            print(atom)
        print("First few atoms in DCD:")
        for atom in traj.topology.atoms[:10]:
            print(atom)
        raise ValueError(f"Mismatch in atom numbers: Trajectory ({traj.topology.n_atoms}) vs PDB ({pdb_topology.n_atoms})")
    
    # Keep only the last fraction of the trajectory
    start_frame = int(traj.n_frames * (1 - fraction))
    traj = traj[start_frame:]
    
    return traj

In [24]:
Lys_Tyr_trajs = [f'/home/bfd21/rds/hpc-work/tbg/cyclization/jobs/md-jobs/Lys-Tyr/results/traj_seed_{x}.dcd' for x in range(10)]

load_trajectory(Lys_Tyr_trajs[0], smiles_dict['Lys-Tyr'])


# Mixture

tolerance 2.000000
filetype pdb
output /tmp/tmpxazb_tf1/tmpp4_1b89v.pdb
add_amber_ter


structure /tmp/tmp4b_w_esl/solute.pdb
  number 1
  inside box 0. 0. 0. 23.000000 23.000000 23.000000
end structure

structure /tmp/tmp4b_w_esl/solvent.pdb
  number 953
  inside box 0. 0. 0. 23.000000 23.000000 23.000000
end structure


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                           Included as part of Packmol Memgen
                                                              Version 20.14.3 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: pdb
  Will add the TER flag between molecul

OSError: The topology is loaded by filename extension, and the detected ".dcd" format is not supported. Supported topology formats include ".pdb", ".pdb.gz", ".h5", ".lh5", ".prmtop", ".parm7", ".prm7", ".psf", ".mol2", ".hoomdxml", ".gro", ".arc", ".hdf5" and ".gsd".